## TODO
- 行数出す
- コード関数化
- WEB共有
- basic認証
- Docker化&インフラ登録＆共有


In [3]:
import base64
import io
import numpy as np
import pandas as pd
from dateutil.relativedelta import relativedelta
import datetime as dt

import plotly.express as px
import plotly.graph_objects as go

from jupyter_dash import JupyterDash 
import dash_html_components as html 
import dash_core_components as dcc 
from dash.dependencies import Input, Output, State
import dash_table


#createする予定データのstartend
data_yotei_start=dt.date(2021, 1,1)
data_yotei_end=dt.date(2022, 12, 31)

core_style={"backgroundColor":"whitesmoke"}

app = JupyterDash(
    __name__,
    suppress_callback_exceptions=True,
)

#関数定義
def Def_Year_Select_createdata(gdf,y,flg):
    createdata_df = gdf
    if flg == 0:
        #print('yyyy',y),
        if y == 'all':
            return createdata_df
            
        if y == '2020':
            return createdata_df[(createdata_df['年月'] > dt.datetime(2020,3,31)) & (createdata_df['年月'] < dt.datetime(2021,4,1))  ]
       
        if y == '2021':
            return createdata_df[(createdata_df['年月'] > dt.datetime(2021,3,31)) & (createdata_df['年月'] < dt.datetime(2022,4,1)) ]
       
        if y == '2022':
            return createdata_df[(createdata_df['年月'] > dt.datetime(2022,3,31)) & (createdata_df['年月'] < dt.datetime(2023,4,1))  ]
           
        if y == '2023':
            return createdata_df[(createdata_df['年月'] > dt.datetime(2023,3,31)) & (createdata_df['年月'] < dt.datetime(2024,4,1))  ]
        
    if flg == 1:
        createdata_df_plan = createdata_df[createdata_df['種別']=='予定']
        createdata_df_fact = createdata_df[createdata_df['種別']=='実績']
        
        if y == 'all':
            return createdata_df_plan,createdata_df_fact
         
        if y == '2020':
            return createdata_df_plan[(createdata_df_plan['年月'] > dt.datetime(2020,3,31)) & (createdata_df_plan['年月'] < dt.datetime(2021,4,1))  ],createdata_df_fact[(createdata_df_fact['年月'] > dt.datetime(2020,3,31)) & (createdata_df_fact['年月'] < dt.datetime(2021,4,1))  ],
      
        if y == '2021':
            return createdata_df_plan[(createdata_df_plan['年月'] > dt.datetime(2021,3,31)) & (createdata_df_plan['年月'] < dt.datetime(2022,4,1)) ],createdata_df_fact[(createdata_df_fact['年月'] > dt.datetime(2021,3,31)) & (createdata_df_fact['年月'] < dt.datetime(2022,4,1))  ],
      
        if y == '2022':
            return createdata_df_plan[(createdata_df_plan['年月'] > dt.datetime(2022,3,31)) & (createdata_df_plan['年月'] < dt.datetime(2023,4,1))  ],createdata_df_fact[(createdata_df_fact['年月'] > dt.datetime(2022,3,31)) & (createdata_df_fact['年月'] < dt.datetime(2023,4,1))  ],
       
        if y == '2023':
            return createdata_df_plan[(createdata_df_plan['年月'] > dt.datetime(2023,3,31)) & (createdata_df_plan['年月'] < dt.datetime(2024,4,1))  ],createdata_df_fact[(createdata_df_fact['年月'] > dt.datetime(2023,3,31)) & (createdata_df_fact['年月'] < dt.datetime(2024,4,1))  ],
            

app.layout = html.Div([
    
    html.H1("FogHorn License",style={"textAlign":"center","backgroundColor":"darkorange"}),
    
    # ➊ Uploadコンポーネントを配置する。
    dcc.Upload(
        id="upload-csv",
        children=html.Div(
            ["Drag and Drop or ", html.A("Select FogHorn license Excel File")]
        ),
        style={
            "width": "80%",
            "height": "80px",
            "lineHeight": "60px",
            "borderWidth": "1px",
            "borderStyle": "dashed",
            "borderRadius": "5px",
            "textAlign": "center",
            "margin": "3% auto",
        },
    ),    
    
    dcc.Dropdown(
        id='selectdown',
        options=[
            {"value" : "income_count" , "label" : "1. ライセンス収入(月毎) & クレジット数" },
            {"value" : "cumsum" , "label" : "2. ライセンス収入(累積) & クレジット数" },
            {"value" : "income" , "label" : "3. ライセンス収入内訳" },  
            {"value" : "count" , "label" : "４. クレジット数内訳" },
        ],
        value= "income_count",
        style=core_style,
    ), 
    
    dcc.RadioItems(
        id='ritem',
        options=[
            {'label': 'all', 'value': 'all'},
            {'label': '2020', 'value': '2020'},
            {'label': '2021', 'value': '2021'},
            {'label': '2022', 'value': '2022'},
            {'label': '2023', 'value': '2023'},
        ],
        value='all'

    ),
    
    dcc.Graph(id='graph'),
    
###################################
    dcc.Dropdown(
        id='selectdown2',
        options=[
            {"value" : "treemap" , "label" : "1. 販売セグメント" },
            {"value" : "sunburst" , "label" : "2. 販売種類" },  
            {"value" : "scatter" , "label" : "3. 販売割引率" },
            {"value" : "guntchart" , "label" : "4. ライセンス期間" },
        ],
        value= "treemap",
        style=core_style,
    ),    

    dcc.Graph(id='graph2'),
  
#####################################   
    
    dcc.Tabs(
            id="tabs",
            value="one",
            children=[
                dcc.Tab(
                label="Table",
                value="one",
                children=[html.Div(id="upload-content")],
                ),
            ]
    ),

    dcc.Tabs(
            id="tabs2",
            value="one",
            children=[
                dcc.Tab(
                label="Table2",
                value="one",
                children=[html.Div(id="upload-content2")],
                ),
            ]
    ),

])

#  テーブルを作成するためのコールバック 1
# df5 create
@app.callback(
    Output("upload-content", "children"),
    Input("upload-csv", "contents"),
    State("upload-csv", "filename"),
)
def update_contents(contents, filename):
    if contents:
        content_type, content_string = contents.split(",")
        decoded = base64.b64decode(content_string)
        # csvファイルとエクセルファイルのデータ読み込みの処理
        try:
            if filename.endswith(".csv"):
                df = pd.read_csv(io.StringIO(decoded.decode("utf-8")))
            elif filename.endswith(".xls") or filename.endswith(".xlsx"):
                df = pd.read_excel(io.BytesIO(decoded))
        except Exception as e:
            print(e)
            return html.Div(["ファイルの処理中にエラーが発生しました"])
        
        #関数定義
        def revenue_per_month(rev1,d1, d2):
            revenue_per_month = rev1/diff_month(d1, d2)
            return revenue_per_month

        def teika_per_month(teika,d1, d2):
            teika_per_month = teika/diff_month(d1, d2)
            return teika_per_month

        def diff_month(d1, d2):
            if d1 > d2:
                d1, d2 = d2, d1
            return (d2.year - d1.year)*12 + d2.month - d1.month + 1

        def month_data_gen(cname,cnt1,rev1,d1, d2, zisseki,teika):
            m = d1
            for i in range(1000):
                if m<d2:
                    l.append([m,cname,cnt1,revenue_per_month(rev1,d1, d2),zisseki,teika_per_month(teika,d1,d2)])
                    #print(m)
                    #print(l)
                    m=m+relativedelta(months=1)
                else:
                    pass
        
        #データクレンジング処理        
        #クレンジングロジック
        #１〜３行目を削除
        df2 = df[4:]

        #４行目の内容をcolumnsへ設定
        df24 = df2.loc[4:4]
        df2.columns = df24.iloc[0].to_list()

        #1行目を削除
        df2 = df2[1:]
        #Nan以外を抽出　
        df2 = df2[~df2['ライセンス発行先'].isnull()]

        #必要列を抽出
        df3 = df2.loc[:,['分類1','分類2','ライセンス発行先','エディション','ライセンス形態','ライセンス数','クレジット数','定価','販売金額','開始期限','終了期限']]

        df3['種別'] = '実績'
        df4 = df3.loc[:,['ライセンス発行先','クレジット数','定価','販売金額','開始期限','終了期限','種別']]
        df5 = df4.append({'ライセンス発行先':'forecast','クレジット数':120,'定価':23400000,'販売金額':23400000,'開始期限':data_yotei_start,'終了期限':data_yotei_end,'種別':'予定'},ignore_index=True)
        #df5 = pd.read_csv('../data/df4to_csv.csv')
        #df5 = df5.drop('Unnamed: 0',axis=1)
        
        df5['開始期限']=pd.to_datetime(df5['開始期限'])
        df5['終了期限']=pd.to_datetime(df5['終了期限'])
        df5['販売金額']=df5['販売金額'].astype(int)
        df5['定価']=df5['定価'].astype(int)
        
        # ファイル名をタイトル、読み込んだデータのテーブルを返り値とする
        return (
            html.H2(filename),
            dash_table.DataTable(
                id="table",
                style_cell={
                    "textAlign": "center",
                    "maxWidth": "80px",
                    "whiteSpace": "normal",
                    "minWidth": "80px",
                },
                fixed_rows={"headers": True, "data": 0},
                style_table={"maxHeight": 800, "maxWidth": "100%"},
                # ➊ セルの編集を可能。
                editable=True,
                # ➊ 列のフィルタリングを可能に。
                filter_action="native",
                # ➊ 行の消去を可能に。
                row_deletable=True,
                # ➊ 複数行の選択を可能に。
                row_selectable="multi",
                # ➊ 列のソートを可能に。
                sort_action="native",
                # ➊ ソートを複数列を条件で可能に。
                sort_mode="multi",
                # ➋ 引数columnsにテーブル作成のデータを渡す
                columns=[
                    {"name": col, "id": col, "deletable": True, "selectable": True}
                    for col in df5.columns
                ],
                data=df5.to_dict("records"),
                # ➌ 1ページのデータを1000行に
                page_size=1000,
                # ➍ データファイルをエクスポート
                export_format="csv",
            ),
        )

    
#datatable return
#df30 create 
@app.callback(
    Output("upload-content2", "children"),
    Input("upload-csv", "contents"),
    State("upload-csv", "filename"),
)
def update_contents(contents, filename):
    if contents:
        content_type, content_string = contents.split(",")
        decoded = base64.b64decode(content_string)
        # csvファイルとエクセルファイルのデータ読み込みの処理
        try:
            if filename.endswith(".csv"):
                df = pd.read_csv(io.StringIO(decoded.decode("utf-8")))
            elif filename.endswith(".xls") or filename.endswith(".xlsx"):
                df = pd.read_excel(io.BytesIO(decoded))
        except Exception as e:
            print(e)
            return html.Div(["ファイルの処理中にエラーが発生しました"])
        
        #関数定義
        def revenue_per_month(rev1,d1, d2):
            revenue_per_month = rev1/diff_month(d1, d2)
            return revenue_per_month

        def teika_per_month(teika,d1, d2):
            teika_per_month = teika/diff_month(d1, d2)
            return teika_per_month

        def diff_month(d1, d2):
            if d1 > d2:
                d1, d2 = d2, d1
            return (d2.year - d1.year)*12 + d2.month - d1.month + 1

        def month_data_gen(cname,cnt1,rev1,d1, d2, zisseki,teika):
            m = d1
            for i in range(1000):
                if m<d2:
                    l.append([m,cname,cnt1,revenue_per_month(rev1,d1, d2),zisseki,teika_per_month(teika,d1,d2)])
                    #print(m)
                    #print(l)
                    m=m+relativedelta(months=1)
                else:
                    pass
        
        #データクレンジング処理        
        #クレンジングロジック
        #１〜３行目を削除
        df2 = df[4:]

        #４行目の内容をcolumnsへ設定
        df24 = df2.loc[4:4]
        df2.columns = df24.iloc[0].to_list()

        #1行目を削除
        df2 = df2[1:]
        #Nan以外を抽出　
        df2 = df2[~df2['ライセンス発行先'].isnull()]

        #必要列を抽出
        df3 = df2.loc[:,['分類1','分類2','ライセンス発行先','エディション','ライセンス形態','ライセンス数','クレジット数','定価','販売金額','開始期限','終了期限']]
   
        #ライセンス分析用データ新規作成 ■
        df30 = df3.loc[:,['ライセンス発行先','クレジット数','定価','販売金額','分類1','分類2','エディション','ライセンス形態','ライセンス数','開始期限','終了期限']]
        #print('df30',df30)
  
        #ライセンス分析用データのデータ加工
        df30['開始期限']=pd.to_datetime(df30['開始期限'])
        df30['終了期限']=pd.to_datetime(df30['終了期限'])
        df30['販売金額']=df30['販売金額'].astype(int)
        df30['定価']=df30['定価'].astype(int)
        
        # ファイル名をタイトル、読み込んだデータのテーブルを返り値とする
        return (
            html.H2(filename),
            dash_table.DataTable(
                id="table2",
                style_cell={
                    "textAlign": "center",
                    "maxWidth": "80px",
                    "whiteSpace": "normal",
                    "minWidth": "80px",
                },
                fixed_rows={"headers": True, "data": 0},
                style_table={"maxHeight": 800, "maxWidth": "100%"},
                # ➊ セルの編集を可能。
                editable=True,
                # ➊ 列のフィルタリングを可能に。
                filter_action="native",
                # ➊ 行の消去を可能に。
                row_deletable=True,
                # ➊ 複数行の選択を可能に。
                row_selectable="multi",
                # ➊ 列のソートを可能に。
                sort_action="native",
                # ➊ ソートを複数列を条件で可能に。
                sort_mode="multi",
                # ➋ 引数columnsにテーブル作成のデータを渡す
                columns=[
                    {"name": col, "id": col, "deletable": True, "selectable": True}
                    for col in df30.columns
                ],
                data=df30.to_dict("records"),
                # ➌ 1ページのデータを1000行に
                page_size=1000,
                # ➍ データファイルをエクスポート
                export_format="csv",
            ),
        )
    
# graph cerate1

@app.callback(
    Output('graph', 'figure'),
    Input('selectdown','value'),
     Input('ritem', 'value'),
     Input("table", "columns"),
     Input("table", "derived_virtual_data"),
)

def update_dropdown(select_values1,selected_values2,columns, rows):
    # テーブルのデータをデータフレームとする

    #関数定義
    def revenue_per_month(rev1,d1, d2):
        revenue_per_month = rev1/diff_month(d1, d2)
        return revenue_per_month

    def teika_per_month(teika,d1, d2):
        teika_per_month = teika/diff_month(d1, d2)
        return teika_per_month

    def diff_month(d1, d2):
        if d1 > d2:
            d1, d2 = d2, d1
        return (d2.year - d1.year)*12 + d2.month - d1.month + 1

    def month_data_gen(cname,cnt1,rev1,d1, d2, zisseki,teika):
        m = d1
        for i in range(1000):
            if m<d2:
                l.append([m,cname,cnt1,revenue_per_month(rev1,d1, d2),zisseki,teika_per_month(teika,d1,d2)])
                #print(m)
                #print(l)
                m=m+relativedelta(months=1)
            else:
                pass
            
    df5 = pd.DataFrame(rows, columns=[c["name"] for c in columns])
    df5['開始期限']=pd.to_datetime(df5['開始期限'])
    df5['終了期限']=pd.to_datetime(df5['終了期限'])
    df5['販売金額']=df5['販売金額'].astype(int)
    df5['定価']=df5['定価'].astype(int)
    
    df_list = df5.values.tolist()
    #print('df_list',df_list)

    l=[]
    for i in range(len(df_list)):
        #print('df_list',df_list[i])

        cname=df_list[i][0]
        cnt1=df_list[i][1]
        rev1=df_list[i][3]
        d1=df_list[i][4]
        d2=df_list[i][5]
        zisseki=df_list[i][6]
        teika=df_list[i][2]

        month_data_gen(cname,cnt1,rev1,d1, d2,zisseki,teika)

    #global createdate_df    
    createdata_df = pd.DataFrame(l)
    createdata_df.columns=['年月','発行先','数','M¥','種別','定価']


    #累積和  事前に予定と実績を分解すること！！
    createdata_df_plan = createdata_df[createdata_df['種別']=='予定']
    #createdata_df_plan
    createdata_df_fact = createdata_df[createdata_df['種別']=='実績']
    #createdata_df_fact

    if select_values1 == 'income_count':
        
        selected_df_csv3 = Def_Year_Select_createdata(createdata_df,selected_values2,0)
    
        fig = px.bar(selected_df_csv3,x='年月',y='M¥',color='種別',barmode='group')
        
        fig.add_trace(
            go.Scatter(
                x=createdata_df[createdata_df['種別'] == '予定']['年月'][~createdata_df[createdata_df['種別'] == '予定']['年月'].duplicated()].sort_values(),
                y=createdata_df[createdata_df['種別'] == '予定'].groupby('年月').sum()['数'], 
                mode='lines+markers',
                name='クレジット数(予定)',
                yaxis='y2',
                marker={'color':'green'},
            )
        )
        
        fig.add_trace(
            go.Scatter(
                x=selected_df_csv3[selected_df_csv3['種別'] == '実績']['年月'][~selected_df_csv3[selected_df_csv3['種別'] == '実績']['年月'].duplicated()].sort_values(),
                y=selected_df_csv3[selected_df_csv3['種別'] == '実績'].groupby('年月').sum()['数'], 
                mode='lines+markers',
                name='クレジット数(実績)',
                yaxis='y2',
                marker={'color':'black'}, 
            )
        )
        
        fig.update_layout(
            title='(result) license income and count',
#             xaxis={
#                 "rangeslider":{"visible":True},
#                 "rangeselector":{
#                     "buttons":[
#                         {"label":"1y","step":"year","count": 1},
#                         {"label":"6m","step":"month","count": 6},
#                         {"step":"all"},
#                     ]
#                 },
#             },
            yaxis1={'title': '収入(¥)', 'showgrid': False,},
            yaxis2={'title': 'クレジット数', 'side': 'right', 'overlaying': 'y', 'showgrid': True,'range':[0,200]},
            bargap=0.3,
        )
        
        return fig

    
    
    
    
########
    if select_values1 == 'cumsum':
    
        selected_df_csv3_plan, selected_df_csv3_fact= Def_Year_Select_createdata(createdata_df,selected_values2,1) 
        
        trace1=go.Bar(
                x=selected_df_csv3_plan.groupby(['年月']).sum().cumsum().index,
                y=selected_df_csv3_plan.groupby(['年月']).sum().cumsum()['M¥'], 
                #mode='lines+markers',
                name='予定',
                yaxis='y1',
                marker={'color':'red'},
            )
        
        
        trace2=(
            go.Bar(
                x=selected_df_csv3_fact.groupby(['年月']).sum().cumsum().index,
                y=selected_df_csv3_fact.groupby(['年月']).sum().cumsum()['M¥'], 
                #mode='bar',
                name='実績',
                yaxis='y1',
                marker={'color':'blue'}, 
                #barmode='group',
            )
        )

            
        trace3=go.Scatter(
                x=createdata_df[createdata_df['種別'] == '予定']['年月'][~createdata_df[createdata_df['種別'] == '予定']['年月'].duplicated()].sort_values(),
                y=createdata_df[createdata_df['種別'] == '予定'].groupby('年月').sum()['数'], 
                mode='lines+markers',
                name='クレジット数(予定)',
                yaxis='y2',
                marker={'color':'green'},
            )
        
        trace4=go.Scatter(
                x=selected_df_csv3_fact[selected_df_csv3_fact['種別'] == '実績']['年月'][~selected_df_csv3_fact[selected_df_csv3_fact['種別'] == '実績']['年月'].duplicated()].sort_values(),
                y=selected_df_csv3_fact[selected_df_csv3_fact['種別'] == '実績'].groupby('年月').sum()['数'], 
                mode='lines+markers',
                name='クレジット数(実績)',
                yaxis='y2',
                marker={'color':'black'},
            )        
        data=[trace1,trace2,trace3,trace4]
        
        layout=go.Layout(
            title='(cumulative) license income and count',
#             xaxis={
#                 "rangeslider":{"visible":True},
#                 "rangeselector":{
#                     "buttons":[
#                         {"label":"1y","step":"year","count": 1},
#                         {"label":"6m","step":"month","count": 6},                        
#                         {"step":"all"},
#                     ]
#                 },
#             },
            yaxis1={'title': '収入', 'showgrid': False,},
            yaxis2={'title': 'クレジット数', 'side': 'right', 'overlaying': 'y', 'showgrid': True},
            bargap=0.3,
            #bargroupgap=0.3,
            barmode='group'
        )
        
        fig = go.Figure(data=data,layout=layout)
        
        return fig
    
    
    
    
########
    if select_values1 == 'income':
        
        selected_df_csv3 = Def_Year_Select_createdata(createdata_df,selected_values2,0)
    
        fig = px.bar(selected_df_csv3[createdata_df['種別'] == '実績'],x='年月',y='M¥',color='発行先')

        fig.add_trace(
            go.Scatter(
                x=createdata_df[createdata_df['種別'] == '予定']['年月'][~createdata_df[createdata_df['種別'] == '予定']['年月'].duplicated()].sort_values(),
                y=createdata_df[createdata_df['種別'] == '予定'].groupby('年月').sum()['M¥'], 
                mode='lines+markers',
                name='予定',
                yaxis='y1',
                marker={'color':'black'},
            )
        )
    
        fig.add_trace(
            go.Scatter(
                x=createdata_df[createdata_df['種別'] == '実績']['年月'][~createdata_df[createdata_df['種別'] == '実績']['年月'].duplicated()].sort_values(),
                y=createdata_df[createdata_df['種別'] == '実績'].groupby('年月').sum()['定価'], 
                mode='lines+markers',
                name='定価',
                yaxis='y1',
                marker={'color':'blue'},
            )
        )
        fig.update_layout(
            title='license income',
            yaxis1={'title': '収入(¥)', 'showgrid': False},
            bargap=0.3,
        )
        return fig
    
    
    
    
########
    if select_values1 == 'count':
        selected_df_csv3 = Def_Year_Select_createdata(createdata_df,selected_values2,0)
            
        fig = px.bar(selected_df_csv3[selected_df_csv3['種別'] == '実績'],x='年月',y='数',color='発行先')   
    
        fig.add_trace(
            go.Scatter(
                x=createdata_df[createdata_df['種別'] == '予定']['年月'][~createdata_df[createdata_df['種別'] == '予定']['年月'].duplicated()].sort_values(),
                y=createdata_df[createdata_df['種別'] == '予定'].groupby('年月').sum()['数'], 
                mode='lines+markers',
                name='予定',
                marker={'color':'green'}, 
            )
        )
        
        fig.update_layout(
            title='license count',
            yaxis1={'title': 'ライセンス数', 'showgrid': False,},
            bargap=0.3,
        )
        
        return fig

#####################################
    
####################    
# graph cerate2
@app.callback(
    Output('graph2', 'figure'),
     Input('selectdown2','value'),
     Input('ritem', 'value'),
     Input("table2", "columns"),
     Input("table2", "derived_virtual_data"),
)

def update_dropdown(select_values1,selected_values2,columns, rows):
    # テーブルのデータをデータフレームとする
    #print("columns",columns)
    #print("rows",rows)

        #関数定義
    def revenue_per_month(rev1,d1, d2):
        revenue_per_month = rev1/diff_month(d1, d2)
        return revenue_per_month

    def teika_per_month(teika,d1, d2):
        teika_per_month = teika/diff_month(d1, d2)
        return teika_per_month

    def diff_month(d1, d2):
        if d1 > d2:
            d1, d2 = d2, d1
        return (d2.year - d1.year)*12 + d2.month - d1.month + 1

    def month_data_gen(cname,cnt1,rev1,d1, d2, zisseki,teika):
        m = d1
        for i in range(1000):
            if m<d2:
                l.append([m,cname,cnt1,revenue_per_month(rev1,d1, d2),zisseki,teika_per_month(teika,d1,d2)])
                m=m+relativedelta(months=1)
            else:
                pass

    df30 = pd.DataFrame(rows, columns=[c["name"] for c in columns])

###########
    if select_values1 == 'guntchart':
        fig = px.timeline(df30, x_start="開始期限", x_end="終了期限", y="ライセンス発行先")
        fig.update_yaxes(autorange="reversed")
        return fig

########
    if select_values1 == 'treemap':
        fig = px.treemap(df30,path=['分類2','分類1','ライセンス発行先'],values='販売金額',hover_name='クレジット数')
        return fig
    
    
########
    if select_values1 == 'sunburst':
        fig = px.sunburst(df30,path=['エディション','ライセンス形態','ライセンス発行先'],values='販売金額',hover_name='クレジット数')
        return fig
    
########
    if select_values1 == 'scatter':
        df30['割引率']=df30['販売金額']/df30['定価']*100
        fig = px.scatter(df30,x='割引率',y='クレジット数',color='ライセンス発行先',size='販売金額',facet_col='分類2')
        return fig


#app.run_server(mode='inline')
#app.run_server(mode='external')
#app.run_server(mode='jupyterlab')
app.run_server(port=8099,debug=False)

Dash app running on http://127.0.0.1:8099/


In [ ]:
#!cloudflared tunnel --url localhost:8080